In [1]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
import glob


#filename queue
filename_queue=glob.glob('./rawdata/data[1-5].csv')
#Create return path
#return_path='./parsed/data.csv'
#create_file=open(return_path,'w')

In [2]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
index=[1,2,3,11,14,15] 
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)

#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        if labl not in index:
            label_count[labl]+=1
            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                    [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])

                #5 features  what else?2
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)

                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)


                #for cross validation
                training_set.append(window_feature)
                training_label.append(labl)


In [4]:
print(np.shape(training_set))

(1144, 180)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X=training_set
y=training_label

clf=RandomForestClassifier()
param_grid={
 'n_estimators':[100,120,180,200]
   # 'max_features':['auto','sqrt','log2'],
    #'max_depth':[10,12,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [6]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [7]:
grid.cv_results_

{'mean_fit_time': array([0.54353805, 0.62748685, 0.94126289, 1.04339738]),
 'std_fit_time': array([0.03370086, 0.00316278, 0.00626297, 0.00895064]),
 'mean_score_time': array([0.00666444, 0.00746644, 0.01078107, 0.01205297]),
 'std_score_time': array([0.00118509, 0.00010011, 0.00017059, 0.00047962]),
 'param_n_estimators': masked_array(data=[100, 120, 180, 200],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 100},
  {'n_estimators': 120},
  {'n_estimators': 180},
  {'n_estimators': 200}],
 'split0_test_score': array([0.83193277, 0.85714286, 0.77310924, 0.79831933]),
 'split1_test_score': array([0.94117647, 0.93277311, 0.93277311, 0.93277311]),
 'split2_test_score': array([0.94871795, 0.98290598, 0.99145299, 0.97435897]),
 'split3_test_score': array([0.88034188, 0.87179487, 0.88888889, 0.8974359 ]),
 'split4_test_score': array([0.92982456, 0.92982456, 0.92982456, 0.92982456]),
 'split5_test_score': array(

In [8]:
grid.best_params_

{'n_estimators': 120}

In [9]:
RF=RandomForestClassifier(120).fit(X,y)

In [10]:
clf1=RandomForestClassifier(180)
scores=cross_val_score(clf1,X,y,cv=10,scoring='accuracy')
print(scores.mean())

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.8852521975026401


In [11]:
from joblib import dump, load
import pickle

filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

['rf.joblib']

In [93]:
#SVM
#prepare the data
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)
scores=cross_val_score(clf,X,y,cv=10,scoring='accuracy')
print(scores.mean())


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.7856916637489739


In [87]:
param_grid={
 #'kernel':['linear','poly','rbf','sigmoid'], rbf to be best acc
    'C':[0.001,0.01,0.1,1,10],
    'gamma' : [0.001, 0.01, 0.1, 1]

}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [88]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.1, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'gamma': [0.001, 0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [89]:
grid.best_params_

{'C': 10, 'gamma': 0.001}

In [90]:
grid.cv_results_

{'mean_fit_time': array([0.34307218, 0.34653304, 0.34341841, 0.32106791, 0.34884019,
        0.32110851, 0.31518092, 0.31136446, 0.33454287, 0.32425928,
        0.31560729, 0.32112675, 0.47151635, 0.50642576, 0.50203469,
        0.51583173, 0.54383953, 0.58650825, 0.49966176, 0.50693669]),
 'std_fit_time': array([0.02549128, 0.03495157, 0.03996245, 0.02927685, 0.0364689 ,
        0.00267671, 0.00752571, 0.00545768, 0.00304246, 0.00493928,
        0.00532335, 0.0202237 , 0.01881682, 0.02975579, 0.03108146,
        0.04618374, 0.03424326, 0.05779977, 0.00503003, 0.03167389]),
 'mean_score_time': array([0.02702363, 0.02937548, 0.03035238, 0.02837565, 0.02750959,
        0.02657707, 0.02628691, 0.02548821, 0.02709322, 0.02734466,
        0.02574279, 0.02686388, 0.02751012, 0.02742758, 0.02914262,
        0.03092718, 0.03417521, 0.03114219, 0.02619624, 0.02743068]),
 'std_score_time': array([0.00072299, 0.00595141, 0.0075207 , 0.00564541, 0.00164436,
        0.00072918, 0.00179509, 0.000726

In [91]:
clf1=sklearn.svm.SVC(gamma='scale',tol=0.1)
scores=cross_val_score(clf1,X,y,cv=10,scoring='accuracy')
print(scores.mean())

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.7856916637489739


In [22]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

ETC=ExtraTreesClassifier(300)
DTC=DecisionTreeClassifier()








In [23]:
param_grid={
 'n_estimators':[100,200,300],
   # 'max_features':['auto','sqrt','log2'],
    'max_depth':[10,14,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=ETC,param_grid=param_grid,cv=10,scoring='accuracy')

In [24]:
grid.fit(X,y)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=None,
                                            max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=300, n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 14, No

In [25]:
grid.cv_results_

{'mean_fit_time': array([0.10543201, 0.21241744, 0.31339355, 0.1079457 , 0.21587787,
        0.32233818, 0.10762014, 0.21365778, 0.36180322]),
 'std_fit_time': array([0.00398968, 0.00862709, 0.00535393, 0.00257703, 0.00246608,
        0.00659796, 0.00099009, 0.00374106, 0.07325796]),
 'mean_score_time': array([0.00628915, 0.01192706, 0.01796401, 0.00621655, 0.01192286,
        0.01773303, 0.0062901 , 0.01191108, 0.02049518]),
 'std_score_time': array([2.80091955e-04, 1.39092265e-04, 1.09014485e-03, 6.13250822e-05,
        1.32504906e-04, 2.21709462e-04, 1.70494264e-04, 9.96613074e-05,
        3.84473639e-03]),
 'param_max_depth': masked_array(data=[10, 10, 10, 14, 14, 14, None, None, None],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[100, 200, 300, 100, 200, 300, 100, 200, 300],
              mask=[False, False, False, False, False, F

In [27]:
test_pred_proba2=ETC.predict_proba(test_set)
array=np.array(test_pred_proba2)
newpred3=np.zeros(len(test_pred4))
for x in range(len(test_pred_proba2)):
    if(np.max(test_pred_proba2[x])>0.2):
        newpred3[x]=test_pred4[x]
    else:
        newpred3[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred3))
print(newpred3)
print(accuracy_score(newpred3,test_label))

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [77]:
from joblib import dump, load
import pickle

filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

['rf.joblib']